# De la normale à la lognormale : l'ajustement de convexité

Ce notebook illustre pourquoi le terme $-\frac{1}{2}\sigma^2$ est nécessaire dans le modèle de prix lognormal.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, lognorm

## Le problème : l'inégalité de Jensen

Pour une variable aléatoire $X$, la fonction exponentielle est **convexe**, donc :
$$E[e^X] \geq e^{E[X]}$$

L'inégalité est stricte lorsque $X$ a une variance positive. Cela signifie :
- Si $X \sim N(\mu, \sigma^2)$, alors $E[e^X] = e^{\mu + \frac{1}{2}\sigma^2}$, **et non** $e^{\mu}$

In [ ]:
# Visualiser la convexité de exp(x)
x = np.linspace(-2, 2, 100)

plt.figure(figsize=(10, 6))
plt.plot(x, np.exp(x), 'b-', linewidth=2, label=r'$e^x$ (fonction convexe)')

# Montrer une corde (sécante) entre deux points
x1, x2 = -1, 1
y1, y2 = np.exp(x1), np.exp(x2)
plt.plot([x1, x2], [y1, y2], 'r--', linewidth=2, label='Corde (sécante)')

# Point milieu
x_mid = (x1 + x2) / 2
y_chord = (y1 + y2) / 2  # Moyenne de e^x1 et e^x2
y_curve = np.exp(x_mid)   # e^(moyenne de x1 et x2)

plt.scatter([x_mid], [y_chord], color='red', s=100, zorder=5, label=r'$\frac{1}{2}(e^{x_1} + e^{x_2})$ = E[$e^X$]')
plt.scatter([x_mid], [y_curve], color='blue', s=100, zorder=5, label=r'$e^{\frac{1}{2}(x_1+x_2)}$ = $e^{E[X]}$')

plt.axvline(x=x_mid, color='gray', linestyle=':', alpha=0.5)
plt.annotate('', xy=(x_mid + 0.05, y_chord), xytext=(x_mid + 0.05, y_curve),
            arrowprops=dict(arrowstyle='<->', color='green', lw=2))
plt.text(x_mid + 0.15, (y_chord + y_curve)/2, 'Écart dû à\nla convexité', fontsize=10, color='green')

plt.xlabel('x', fontsize=12)
plt.ylabel(r'$e^x$', fontsize=12)
plt.title(r"Inégalité de Jensen : $E[e^X] > e^{E[X]}$ pour les fonctions convexes", fontsize=14)
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)
plt.show()

## Simulation : comparaison de $E[e^X]$ vs $e^{E[X]}$

Simulons et vérifions que pour $X \sim N(\mu, \sigma^2)$ :
$$E[e^X] = e^{\mu + \frac{1}{2}\sigma^2}$$

In [ ]:
# Paramètres
mu = 0.10      # Moyenne de X
sigma = 0.30   # Écart-type de X
n_simulations = 100000

# Simuler X ~ N(mu, sigma^2)
X = np.random.normal(mu, sigma, n_simulations)

# Calculer e^X
eX = np.exp(X)

# Comparer
print("Résultats de simulation :")
print(f"  E[X]     = {np.mean(X):.6f}  (vrai : {mu})")
print(f"  e^E[X]   = {np.exp(np.mean(X)):.6f}  (vrai : {np.exp(mu):.6f})")
print(f"  E[e^X]   = {np.mean(eX):.6f}  (vrai : {np.exp(mu + 0.5*sigma**2):.6f})")
print()
print(f"L'écart : E[e^X] - e^E[X] = {np.mean(eX) - np.exp(np.mean(X)):.6f}")
print(f"Cela équivaut à e^mu * (e^(sigma^2/2) - 1) = {np.exp(mu) * (np.exp(0.5*sigma**2) - 1):.6f}")

## Application aux prix des actions

Si les log-rendements sont normaux : $\ln(S_T/S_0) \sim N(m, \sigma^2 T)$

Alors : $S_T = S_0 \cdot e^{\ln(S_T/S_0)}$

Et : $E[S_T] = S_0 \cdot e^{m + \frac{1}{2}\sigma^2 T}$

**Si on veut** $E[S_T] = S_0 \cdot e^{\alpha T}$ (où $\alpha$ est le rendement espéré), on doit poser :
$$m = \left(\alpha - \frac{1}{2}\sigma^2\right) T$$

In [ ]:
# Simulation du prix de l'action
S0 = 100        # Prix initial
alpha = 0.10    # Rendement espéré (10%)
sigma = 0.30    # Volatilité (30%)
T = 1           # 1 an
n_simulations = 100000

# INCORRECT : Utiliser alpha*T comme moyenne des log-rendements
log_returns_wrong = np.random.normal(alpha * T, sigma * np.sqrt(T), n_simulations)
S_T_wrong = S0 * np.exp(log_returns_wrong)

# CORRECT : Utiliser (alpha - 0.5*sigma^2)*T comme moyenne des log-rendements
log_returns_correct = np.random.normal((alpha - 0.5*sigma**2) * T, sigma * np.sqrt(T), n_simulations)
S_T_correct = S0 * np.exp(log_returns_correct)

print("Prix espéré cible : S0 * e^(alpha*T) = %.4f" % (S0 * np.exp(alpha * T)))
print()
print("SANS ajustement de convexité (moyenne du log-rendement = alpha*T) :")
print(f"  E[S_T] = {np.mean(S_T_wrong):.4f}")
print(f"  Rendement implicite = {np.log(np.mean(S_T_wrong)/S0)/T:.4f} (devrait être {alpha})")
print()
print("AVEC ajustement de convexité (moyenne du log-rendement = (alpha - 0.5*sigma^2)*T) :")
print(f"  E[S_T] = {np.mean(S_T_correct):.4f}")
print(f"  Rendement implicite = {np.log(np.mean(S_T_correct)/S0)/T:.4f} (devrait être {alpha})")

In [ ]:
# Visualiser les deux distributions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gauche : Log-rendements (Normal)
ax1 = axes[0]
bins = np.linspace(-1, 1.5, 50)
ax1.hist(log_returns_wrong, bins=bins, density=True, alpha=0.5, label=f'Moyenne = $\\alpha T$ = {alpha*T:.2f}', color='red')
ax1.hist(log_returns_correct, bins=bins, density=True, alpha=0.5, label=f'Moyenne = $(\\alpha - \\frac{{1}}{{2}}\\sigma^2)T$ = {(alpha-0.5*sigma**2)*T:.4f}', color='blue')
ax1.axvline(x=alpha*T, color='red', linestyle='--', linewidth=2)
ax1.axvline(x=(alpha - 0.5*sigma**2)*T, color='blue', linestyle='--', linewidth=2)
ax1.set_xlabel('Log-rendement', fontsize=12)
ax1.set_ylabel('Densité', fontsize=12)
ax1.set_title('Distribution des log-rendements (normale)', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Droite : Prix (Lognormal)
ax2 = axes[1]
bins = np.linspace(20, 250, 50)
ax2.hist(S_T_wrong, bins=bins, density=True, alpha=0.5, label=f'E[$S_T$] = {np.mean(S_T_wrong):.2f} (incorrect)', color='red')
ax2.hist(S_T_correct, bins=bins, density=True, alpha=0.5, label=f'E[$S_T$] = {np.mean(S_T_correct):.2f} (correct)', color='blue')
ax2.axvline(x=np.mean(S_T_wrong), color='red', linestyle='--', linewidth=2)
ax2.axvline(x=np.mean(S_T_correct), color='blue', linestyle='--', linewidth=2)
ax2.axvline(x=S0*np.exp(alpha*T), color='green', linestyle='-', linewidth=2, label=f'Cible : $S_0 e^{{\\alpha T}}$ = {S0*np.exp(alpha*T):.2f}')
ax2.set_xlabel('Prix de l\'action $S_T$', fontsize=12)
ax2.set_ylabel('Densité', fontsize=12)
ax2.set_title('Distribution des prix (lognormale)', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## L'ajustement de convexité en fonction de la volatilité

L'ajustement $-\frac{1}{2}\sigma^2$ croît de manière quadratique avec la volatilité.

In [ ]:
# Montrer comment l'ajustement varie avec sigma
sigmas = np.linspace(0, 0.8, 100)
adjustment = -0.5 * sigmas**2

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gauche : L'ajustement lui-même
ax1 = axes[0]
ax1.plot(sigmas * 100, adjustment * 100, 'b-', linewidth=2)
ax1.fill_between(sigmas * 100, adjustment * 100, 0, alpha=0.3)
ax1.set_xlabel('Volatilité $\\sigma$ (%)', fontsize=12)
ax1.set_ylabel('Ajustement de convexité (%)', fontsize=12)
ax1.set_title('L\'ajustement de convexité $-\\frac{1}{2}\\sigma^2$', fontsize=14)
ax1.grid(True, alpha=0.3)

# Ajouter quelques points de référence
for s in [0.2, 0.4, 0.6]:
    adj = -0.5 * s**2
    ax1.scatter([s*100], [adj*100], color='red', s=80, zorder=5)
    ax1.annotate(f'$\\sigma$={s*100:.0f}% : adj={adj*100:.1f}%', 
                xy=(s*100, adj*100), xytext=(s*100+5, adj*100-2),
                fontsize=10)

# Droite : Moyenne du log-rendement vs rendement espéré
ax2 = axes[1]
alpha = 0.10
ax2.axhline(y=alpha*100, color='green', linestyle='-', linewidth=2, label=f'Rendement espéré $\\alpha$ = {alpha*100}%')
ax2.plot(sigmas * 100, (alpha + adjustment) * 100, 'b-', linewidth=2, label=r'Moyenne du log-rendement = $\alpha - \frac{1}{2}\sigma^2$')
ax2.fill_between(sigmas * 100, alpha * 100, (alpha + adjustment) * 100, alpha=0.3, label='Écart (ajustement de convexité)')
ax2.set_xlabel('Volatilité $\\sigma$ (%)', fontsize=12)
ax2.set_ylabel('Taux (%)', fontsize=12)
ax2.set_title('Rendement espéré vs moyenne du log-rendement', fontsize=14)
ax2.legend(loc='lower left')
ax2.grid(True, alpha=0.3)
ax2.set_ylim(-15, 15)

plt.tight_layout()
plt.show()

## Point clé à retenir

| Quantité | Formule |
|----------|--------|
| Moyenne du log-rendement | $(\alpha - \delta - \frac{1}{2}\sigma^2)T$ |
| Rendement espéré (composé en continu) | $\alpha$ |
| Gain en capital espéré | $\alpha - \delta$ |

Le terme $-\frac{1}{2}\sigma^2$ n'est **pas** une pénalité ou un frein aux rendements. C'est simplement un ajustement comptable qui assure que le paramètre $\alpha$ est égal au rendement espéré.

**Sans l'ajustement :** Le paramètre dans la moyenne serait difficile à interpréter.

**Avec l'ajustement :** $\alpha$ représente directement le rendement espéré, ce qui facilite le passage à l'évaluation risque-neutre en posant $\alpha = r$.

In [ ]:
# Visualisation finale : la distribution lognormale et sa moyenne
S0 = 100
alpha = 0.10
sigma = 0.30
T = 1

# Paramètres pour la lognormale
mu_log = np.log(S0) + (alpha - 0.5*sigma**2) * T
sigma_log = sigma * np.sqrt(T)

# Créer l'intervalle de prix
S = np.linspace(1, 250, 500)

# PDF lognormale
pdf = lognorm.pdf(S, s=sigma_log, scale=np.exp(mu_log))

# Statistiques clés
E_S = S0 * np.exp(alpha * T)  # Espérance
median_S = np.exp(mu_log)      # Médiane (= exp de la moyenne du log)
mode_S = np.exp(mu_log - sigma_log**2)  # Mode

plt.figure(figsize=(12, 6))
plt.plot(S, pdf, 'b-', linewidth=2, label='PDF lognormale')
plt.fill_between(S, pdf, alpha=0.2)

plt.axvline(x=mode_S, color='purple', linestyle='--', linewidth=2, label=f'Mode = {mode_S:.2f}')
plt.axvline(x=median_S, color='orange', linestyle='--', linewidth=2, label=f'Médiane = {median_S:.2f}')
plt.axvline(x=E_S, color='red', linestyle='-', linewidth=2, label=f'Moyenne = {E_S:.2f}')
plt.axvline(x=S0, color='gray', linestyle=':', linewidth=2, label=f'Initial $S_0$ = {S0}')

plt.xlabel('Prix de l\'action $S_T$', fontsize=12)
plt.ylabel('Densité', fontsize=12)
plt.title(f'Distribution lognormale ($\\alpha$={alpha*100}%, $\\sigma$={sigma*100}%, T={T} an)', fontsize=14)
plt.legend(loc='upper right')
plt.grid(True, alpha=0.3)
plt.xlim(0, 250)

# Ajouter une annotation sur l'asymétrie
plt.annotate('Mode < Médiane < Moyenne\n(asymétrie positive)', 
            xy=(E_S, 0.002), xytext=(180, 0.012),
            fontsize=11, 
            arrowprops=dict(arrowstyle='->', color='black'),
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.show()

print(f"Mode = exp(mu - sigma^2) = {mode_S:.4f}")
print(f"Médiane = exp(mu) = {median_S:.4f}")
print(f"Moyenne = exp(mu + sigma^2/2) = S0 * exp(alpha*T) = {E_S:.4f}")
print(f"\nLa moyenne est tirée vers la droite par l'asymétrie positive (convexité de exp).")